<a href="https://colab.research.google.com/github/subhamulal/image-classification-/blob/main/Subham_Ulal_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install unrar
!pip install rarfile


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [ ]:
import rarfile

rar_path = "/content/3_class_tumor.rar"
extract_path = "/content/extracted"  # Path where images will be extracted

rar = rarfile.RarFile(rar_path)
rar.extractall(extract_path)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [ ]:
img_size = (224, 224)
batch_size = 32

In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

In [ ]:
train_generator = datagen.flow_from_directory(
    extract_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # 🔥 Ensures one-hot encoding
    subset='training'
)

val_generator = datagen.flow_from_directory(
    extract_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # 🔥 Ensures one-hot encoding
    subset='validation'
)


Found 2452 images belonging to 1 classes.
Found 612 images belonging to 1 classes.


In [ ]:
for images, labels in train_generator:
    print("Sample label shape:", labels.shape)
    print("Labels:", labels[:5])  # Print first 5 labels
    break

Sample label shape: (32, 1)
Labels: [[1.]
 [1.]
 [1.]
 [1.]
 [1.]]


In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

In [ ]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')  # ✅ Ensure 3 output neurons
])

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy','f1_score'])


In [ ]:
history = model.fit(train_generator, epochs=5, validation_data=val_generator)

Epoch 1/5
77/77 ━━━━━━━━━━━━━━━━━━━━ 544s 7s/step - accuracy: 1.0000 - f1_score: 0.3333 - loss: 7.4409e-07 - val_accuracy: 1.0000 - val_f1_score: 0.3333 - val_loss: 0.0000e+00
Epoch 2/5
77/77 ━━━━━━━━━━━━━━━━━━━━ 534s 7s/step - accuracy: 1.0000 - f1_score: 0.3333 - loss: 5.8185e-09 - val_accuracy: 1.0000 - val_f1_score: 0.3333 - val_loss: 0.0000e+00
Epoch 3/5
77/77 ━━━━━━━━━━━━━━━━━━━━ 527s 7s/step - accuracy: 1.0000 - f1_score: 0.3333 - loss: 5.6210e-09 - val_accuracy: 1.0000 - val_f1_score: 0.3333 - val_loss: 0.0000e+00
Epoch 4/5
77/77 ━━━━━━━━━━━━━━━━━━━━ 529s 7s/step - accuracy: 1.0000 - f1_score: 0.3333 - loss: 3.2122e-09 - val_accuracy: 1.0000 - val_f1_score: 0.3333 - val_loss: 0.0000e+00
Epoch 5/5
77/77 ━━━━━━━━━━━━━━━━━━━━ 565s 7s/step - accuracy: 1.0000 - f1_score: 0.3333 - loss: 2.7595e-09 - val_accuracy: 1.0000 - val_f1_score: 0.3333 - val_loss: 0.0000e+00


In [ ]:
model.save("brain_tumor_classification.h5")
print("Model saved as brain_tumor_classification.h5")

Model saved as brain_tumor_classification.h5
